# Data Cleaning – Real-Time Cruise Ship Data
This notebook cleans and standardizes real-time scraped cruise ship data to create an analysis-ready dataset for downstream EDA and dashboards.


In [1]:
import pandas as pd
import numpy as np
import re

In [6]:
# load raw data
df = pd.read_csv("../data/cruise_ships.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    1566 non-null   object 
 1   year_built              1566 non-null   int64  
 2   age                     1458 non-null   float64
 3   last_refurbishment      821 non-null    float64
 4   gross_tonnage           1362 non-null   float64
 5   decks                   1538 non-null   object 
 6   decks_with_cabins       742 non-null    float64
 7   passengers              1555 non-null   object 
 8   crew                    1439 non-null   float64
 9   cabins                  1455 non-null   float64
 10  engines                 454 non-null    object 
 11  sister_ships            1071 non-null   object 
 12  christened_by           744 non-null    object 
 13  operator                1526 non-null   object 
 14  owner                   1547 non-null   

In [3]:
# drop unusable columns
cols_to_drop = [
    "itinerary_block",
    "propulsion_power_mw"
]
# Filter columns that actually exist in the dataframe
existing_cols_to_drop = [c for c in cols_to_drop if c in df.columns]
df = df.drop(columns=existing_cols_to_drop)
print(f"Dropped columns: {existing_cols_to_drop}")

# Clean passengers column - convert various formats to numeric values
def passengers_to_number(value):
    """Convert passengers column values to numeric, handling various formats."""
    if pd.isna(value):
        return np.nan
    
    # Convert to string and clean
    s = str(value).replace(",", "").strip()
    
    # Handle range formats like "14-28" or "1000–1200" by taking midpoint
    m = re.match(r"(\d+)\s*[-–]\s*(\d+)", s)
    if m:
        try:
            return (int(m.group(1)) + int(m.group(2))) / 2
        except (ValueError, TypeError):
            return np.nan
    
    # Handle plain numbers
    if s.isdigit():
        try:
            return float(s)
        except (ValueError, TypeError):
            return np.nan
    
    return np.nan

# Apply the cleaning function
df["passengers_clean"] = df["passengers"].apply(passengers_to_number)

# Validate the transformation
print(f"Passengers cleaning summary:")
print(f"  Original non-null count: {df['passengers'].notna().sum()}")
print(f"  Cleaned non-null count: {df['passengers_clean'].notna().sum()}")

# Drop original passengers column
df = df.drop(columns=["passengers"])

# Clean decks column - extract numeric values only
df["decks_clean"] = (
    df["decks"]
    .astype(str)
    .str.extract(r"(\d+)")  # Extract first sequence of digits
    .astype(float)
)

# Validate decks cleaning
print(f"\nDecks cleaning summary:")
print(f"  Original non-null count: {df['decks'].notna().sum()}")
print(f"  Cleaned non-null count: {df['decks_clean'].notna().sum()}")

# Drop original decks column
df = df.drop(columns=["decks"])

# Clean and standardize categorical columns
categorical_cols = ["operator", "owner", "flag_state", "class", "builder"]
for col in categorical_cols:
    if col in df.columns:
        # Apply string cleaning and title case
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()  # Remove whitespace
            .str.title()  # Convert to Title Case
            .replace("Nan", np.nan)  # Convert string "Nan" back to actual NaN
        )
        print(f"Cleaned categorical column: {col}")

# Convert scrape_time to datetime
df["scrape_time"] = pd.to_datetime(df["scrape_time"], errors="coerce")

# Print summary of datetime conversion
print(f"\nScrape time conversion summary:")
print(f"  Successfully converted: {df['scrape_time'].notna().sum()} out of {len(df)}")
print(f"  Date range: {df['scrape_time'].min()} to {df['scrape_time'].max()}")

# Display cleaned dataframe info
print("\n" + "="*50)
print("CLEANED DATAFRAME INFO:")
print("="*50)
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn dtypes:")
print(df.dtypes)

Dropped columns: ['itinerary_block', 'propulsion_power_mw']
Passengers cleaning summary:
  Original non-null count: 1555
  Cleaned non-null count: 1528

Decks cleaning summary:
  Original non-null count: 1538
  Cleaned non-null count: 1519
Cleaned categorical column: operator
Cleaned categorical column: owner
Cleaned categorical column: flag_state
Cleaned categorical column: class
Cleaned categorical column: builder

Scrape time conversion summary:
  Successfully converted: 1566 out of 1566
  Date range: 2025-12-19 22:53:36.705764+00:00 to 2025-12-20 00:53:01.651520+00:00

CLEANED DATAFRAME INFO:
Total rows: 1566
Total columns: 25

DataFrame shape: (1566, 25)

Column dtypes:
name                                   object
year_built                              int64
age                                   float64
last_refurbishment                    float64
gross_tonnage                         float64
decks_with_cabins                     float64
crew                                  fl

In [5]:
# split datasets
df_eda = df.copy()
df_ml = df.dropna(subset=["crew"])

# EDA dataset
eda_cols = [
    "name",
    "year_built",
    "age",
    "gross_tonnage",
    "passengers_clean",
    "crew",
    "cabins",
    "length_m",
    "beam_m",
    "speed_kn",
    "decks_clean",
    "decks_with_cabins",
    "operator",
    "owner",
    "flag_state",
    "class",
    "builder",
    "has_itinerary",
    "scrape_time",
    "building_cost_million",
    "building_cost_currency"
]
df_eda = df_eda[[c for c in eda_cols if c in df_eda.columns]]
# ML dataset
ml_cols = [
    "name",
    "year_built",
    "age",
    "gross_tonnage",
    "passengers_clean",
    "crew",
    "cabins",
    "length_m",
    "beam_m",
    "speed_kn",
    "decks_clean",
    "operator",
    "owner",
    "flag_state",
    "class",
    "builder"
]
df_ml = df_ml[[c for c in ml_cols if c in df_ml.columns]]
# save outputs
df_eda.to_csv("../data/cruise_ships_eda.csv", index=False)
df_ml.to_csv("../data/cruise_ships_ml.csv", index=False)
df_eda.head()

,name,year_built,age,gross_tonnage,passengers_clean,crew,cabins,length_m,beam_m,speed_kn,...,decks_with_cabins,operator,owner,flag_state,class,builder,has_itinerary,scrape_time,building_cost_million,building_cost_currency
0,50 Let Pobedy icebreaker,2007,18.0,23440.0,96.0,140.0,64.0,151.0,30.0,10.2,...,NaN,"Rosatom (Rosmorport), Quark Expeditions, Noble...",Russian Federation (Via Fsue Atomflot),Russia,"Russian Nuclear Icebreaker (Arktika-Class, Pro...",Baltiysky Zavod/Baltic Shipyard (St Petersburg...,False,2025-12-19 22:53:36.705764+00:00,NaN,NaN
1,Abel Matutes ferry,2010,15.0,29670.0,900.0,NaN,92.0,190.0,26.0,23.0,...,NaN,Balearia Ferries Review Wiki Abel Matutes Ferr...,Balearia Group,Malta,Cruiseferry (Ro-Pax Ship) Ferry Route / Homepo...,"Barreras Shipyard (Vigo, Spain)",False,2025-12-19 22:53:40.495211+00:00,NaN,NaN
2,ACL American Anthem,2027,NaN,5148.0,180.0,43.0,94.0,82.0,17.0,12.0,...,NaN,American Cruise Lines Itineraries Review Wiki ...,American Cruise Lines Inc,NaN,Acl Modern Riverboat,"Chesapeake Shipbuilding (Salisbury, Maryland Usa)",True,2025-12-19 22:53:44.387820+00:00,30.0,USD
3,ACL American Constellation,2017,8.0,2400.0,175.0,43.0,90.0,82.0,17.0,12.0,...,NaN,American Cruise Lines Itineraries Review Wiki ...,American Cruise Lines Inc,Usa,Acl Coastal Cruiser Usa,"Chesapeake Shipbuilding (Salisbury, Maryland Usa)",True,2025-12-19 22:53:48.378982+00:00,30.0,USD
4,ACL American Constitution,2018,7.0,2400.0,170.0,43.0,90.0,82.0,17.0,12.0,...,NaN,American Cruise Lines Itineraries Review Wiki ...,American Cruise Lines Inc,Usa,Acl Coastal Cruiser Usa,"Chesapeake Shipbuilding (Salisbury, Maryland Usa)",True,2025-12-19 22:53:52.481941+00:00,30.0,USD
